In [ ]:
import requests
import json
import re
import hashlib

In [ ]:
GITHUB_TOKEN = "####"

MAX_SAMPLES = 100
LANGUAGES = ["python", "java"]

VALID_LICENSES = ["mit", "apache-2.0", "bsd-3-clause", "bsd-2-clause"]

headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

dataset = []
seen_hashes = set()

In [ ]:
# hashing to avoid duplicates
def hash_text(text):
    return hashlib.md5(text.encode("utf-8")).hexdigest()


In [ ]:
# GitHub search

def search_github_files(language, page=1):
    query = f"language:{language} def in:file" if language == "python" else f"language:{language} in:file"
    url = f"https://api.github.com/search/code?q={query}&page={page}&per_page=50"
    return requests.get(url, headers=headers).json()



In [ ]:
# Fetch repo license

def get_repo_license(repo_full_name):
    url = f"https://api.github.com/repos/{repo_full_name}/license"
    res = requests.get(url, headers=headers).json()
    if "license" in res and res["license"]:
        return res["license"]["spdx_id"].lower()
    return None


In [ ]:
# Download raw code

def download_raw_file(item):
    url = item["html_url"].replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")
    res = requests.get(url, headers=headers)
    return res.text if res.status_code == 200 else None



In [ ]:
# Filtering rules to avoid getting ai generated comments

BANNERS = ["====", "----", "****", "#####", "{{{", "}}}", "========"]
VERB_KEYWORDS = [
    "return", "compute", "calculate", "parse", "load", "read", "build",
    "convert", "check", "verify", "open", "write", "download",
    "initialize", "process", "retrieve", "extract"
]


def is_valid_comment(comment):
    if len(comment.strip()) == 0:
        return False

    if any(b in comment for b in BANNERS):
        return False

    if len(comment.split("\n")) > 10:
        return False  # too big = probably file header

    # Require at least one meaningful verb
    if not any(v in comment.lower() for v in VERB_KEYWORDS):
        return False

    return True



In [ ]:
# Extract Python docstrings + function signature

def extract_python(code):
    samples = []

    # Regex for Python docstring + function
    pattern = r'def\s+(\w+)\s*\([^)]*\)\s*:[\s\n]*("""([\s\S]*?)"""|\'\'\'([\s\S]*?)\'\'\')'
    matches = re.finditer(pattern, code)

    for m in matches:
        func_name = m.group(1)
        raw_comment = m.group(3) or m.group(4)
        full_match = m.group(0)

        if not is_valid_comment(raw_comment):
            continue

        # function size filter
        lines = full_match.strip().split("\n")
        if len(lines) < 3 or len(lines) > 80:
            continue

        samples.append({
            "language": "python",
            "human_comment": raw_comment.strip(),
            "code": full_match.strip()
        })

    return samples



In [ ]:
# Extract Java Javadoc + method signature

def extract_java(code):
    samples = []

    pattern = r'(/\*\*[\s\S]*?\*/)\s*(public|private|protected)?\s+\w[\w<>]*\s+\w+\([^)]*\)\s*\{'
    matches = re.finditer(pattern, code)

    for m in matches:
        comment = m.group(1)
        code_snippet = m.group(0)

        if not is_valid_comment(comment):
            continue

        # function size filter
        lines = code_snippet.strip().split("\n")
        if len(lines) < 3 or len(lines) > 80:
            continue

        samples.append({
            "language": "java",
            "human_comment": comment.strip(),
            "code": code_snippet.strip()
        })

    return samples



In [1]:
# main data collection loop

def collect():
    global dataset

    for lang in LANGUAGES:
        print(f"\n🔍 Searching {lang} repositories...")

        page = 1
        while len(dataset) < MAX_SAMPLES:
            results = search_github_files(lang, page)

            if "items" not in results:
                print(" ! API rate limit or no more results.")
                return

            for item in results["items"]:
                repo_name = item["repository"]["full_name"]

                # Check repo license
                license_key = get_repo_license(repo_name)
                if license_key not in VALID_LICENSES:
                    continue

                # Download file raw content
                code = download_raw_file(item)
                if not code:
                    continue

                # Extract relevant code-comment pairs
                if lang == "python":
                    extracted = extract_python(code)
                else:
                    extracted = extract_java(code)

                for s in extracted:
                    # Deduplication
                    h = hash_text(s["code"] + s["human_comment"])
                    if h in seen_hashes:
                        continue
                    seen_hashes.add(h)

                    # Add metadata
                    s["license"] = license_key
                    s["source_repo"] = f"https://github.com/{repo_name}"
                    dataset.append(s)

                    print(f"✔ Sample {len(dataset)} added")

                    if len(dataset) >= MAX_SAMPLES:
                        return

            page += 1


# RUN SCRIPT

collect()

with open("clean_dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=4, ensure_ascii=False)

print("\n DONE ! ")
print(f"Collected {len(dataset)} clean, meaningful samples → clean_dataset.json")



🔍 Searching python repositories...
✔ Sample 1 added
✔ Sample 2 added
✔ Sample 3 added
✔ Sample 4 added
✔ Sample 5 added
✔ Sample 6 added
✔ Sample 7 added
✔ Sample 8 added
✔ Sample 9 added
✔ Sample 10 added
✔ Sample 11 added
✔ Sample 12 added
✔ Sample 13 added
✔ Sample 14 added
✔ Sample 15 added
✔ Sample 16 added
✔ Sample 17 added
✔ Sample 18 added
✔ Sample 19 added
✔ Sample 20 added
✔ Sample 21 added
✔ Sample 22 added
✔ Sample 23 added
✔ Sample 24 added
✔ Sample 25 added
✔ Sample 26 added
✔ Sample 27 added
✔ Sample 28 added
✔ Sample 29 added
✔ Sample 30 added
✔ Sample 31 added
✔ Sample 32 added
✔ Sample 33 added
✔ Sample 34 added
✔ Sample 35 added
✔ Sample 36 added
✔ Sample 37 added
✔ Sample 38 added
✔ Sample 39 added
✔ Sample 40 added
✔ Sample 41 added
✔ Sample 42 added
✔ Sample 43 added
✔ Sample 44 added
✔ Sample 45 added
✔ Sample 46 added
✔ Sample 47 added
✔ Sample 48 added
✔ Sample 49 added
✔ Sample 50 added
✔ Sample 51 added
✔ Sample 52 added
✔ Sample 53 added
✔ Sample 54 added
✔

In [2]:
# write dataset_description.md

description = f"""
# Dataset Description

This dataset contains {len(dataset)} high–quality code–comment pairs
automatically extracted from open–source GitHub repositories.

## Key Features
- Languages: Python and Java
- Only human-written function-level comments (docstrings / Javadoc)
- Functions filtered for clarity and meaningful documentation
- Fully open-source licenses (MIT, Apache-2.0, BSD)
- Deduplicated and quality-checked samples
- Extracted using automated GitHub API pipeline

## Files
- `clean_dataset.json` — The dataset containing all collected samples
- `dataset_description.md` — This file (short description)

## Purpose
Designed for training and evaluating AI models that generate or improve
code comments from real-world open-source projects.

"""

with open("dataset_description.md", "w", encoding="utf-8") as f:
    f.write(description.strip())
